In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
from urllib.parse import parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

In [2]:
from IPython.display import Image
Image(url="https://static.tildacdn.com/tild3361-3935-4166-a132-616437623963/IGF-Thumbnail.png", width=300, height=200)

In [3]:
#
# YouTubeTranscriptApi.get_transcript("bAtxYDZsHW8", languages=['ru', 'en'])

Сессии и спикеры Internet Governance Forum Russia 2010—2021

## Данные о сессиях IGF 2021-2015

In [4]:
def parse_igf_v2(soup, year):

    for report in soup.select("tr > td#left"):

        video_page = report.select(".arrow_presentation a")
        video_url = None
        if len(video_page) > 0:
            video_url = video_page[0]['href']
            if "?p=video" not in video_url:
                video_url = None
            else:
                video_url = parse_qs(video_url)['vid'][0]
                video_url = None if len(video_url) != 11 else video_url
    

        yield {
            'youtube_id': video_url,
            'persons': [p.text for p in report.select("ul li a")],
            'title': report.find("h4").text,
            'summary': report.find("p").text if report.find("p") else None,
            'year': year,
        }

In [5]:
reports = []

In [6]:
sessions_v2 = {
    2021: "https://rigf2021.ru/prog/?p=prog",
    # 2020: "https://rigf2020.ru/prog/?p=prog",
    2019: "https://rigf2019.ru/prog/?p=prog",
    2018: "https://rigf2018.ru/prog/?p=prog",
    2017: "https://rigf2017.ru/prog/?p=prog",
    2016: "https://rigf2016.ru/prog/?p=prog",
    2015: "https://rigf2015.ru/prog/?p=prog",
}

In [7]:
for (year, link) in sessions_v2.items():
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    for r in parse_igf_v2(soup, year): 
        reports.append(r)

## Данные о сессиях IGF 2014-2011

In [8]:
def parse_igf_v1(soup, year):

    for report in soup.select("table.pro td"):

        if report.find("h3") is None:
            continue

        video_page = report.select(".arrow a")
        video_url = None
        if len(video_page) > 0:
            video_url = video_page[len(video_page)-1]['href']
            if "?p=video" not in video_url:
                video_url = None
            else:
                video_url = parse_qs(video_url)['vid'][0]
                video_url = None if len(video_url) != 11 else video_url

        yield {
            'youtube_id': video_url,
            'persons': [p.text for p in report.select("ul li a")],
            'title': report.find("h3").text,
            'summary': report.find("p").text if report.find("p") else None,
            'year': year,
        }

In [9]:
sessions_v1 = {
    2014: "https://rigf2014.ru/prog/?p=prog",
    2013: "https://rigf2013.ru/prog/?p=prog",
    2012: "https://rigf2012.ru/prog/?p=prog",
    2011: "https://rigf2011.ru/prog/?p=prog",
}

In [10]:
for (year, link) in sessions_v1.items():
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    for r in parse_igf_v1(soup, year):
        reports.append(r)

## Данные о сессии IGF 2010

In [11]:
def parse_igf_v0(soup, year):
    for report in soup.select("tr > td#left"):
        yield {
            'youtube_id': None,
            'persons': [p.text for p in report.select("ul li strong a")],
            'title': report.find("strong a").text,
            'summary': report.find("p").text if report.find("p") else None,
            'year': year,
        }

In [12]:
igfru2010_link = "https://rigf2010.ru/eng/program.php"
page = requests.get(link)
soup = BeautifulSoup(page.text, 'html.parser')
for r in parse_igf_v0(soup, 2010): reports.append(r)

## Транскрипции выступлений

In [13]:
for r in reports:
    try:
        transcript = YouTubeTranscriptApi.get_transcript(r['youtube_id'], languages=['ru'])
        r['video_transcript'] = " ".join([t['text'] for t in transcript])
    except:
        r['video_transcript'] = None

In [14]:
for r in reports:
    persons = r['persons']
    persons = [p.strip() for p in persons]
    persons = [p.replace("\n", " ") for p in persons]
    persons = [p.replace("\r", " ") for p in persons]
    persons = [p.replace("\xa0", " ") for p in persons]
    persons = [p.replace("  ", " ") for p in persons]
    persons = [' '.join(p.split()) for p in persons]
    r['persons'] = persons

## Все сессии и спикеры

In [15]:
df = pd.DataFrame.from_dict(reports)
df

,youtube_id,persons,title,summary,year,video_transcript
0,248b1oXAvB0,"[Максут Шадаев, Александр Хинштейн, Татьяна Ма...",Церемония открытия,6 Всероссийский молодежный конкурс работ по п...,2021,друзья доброе утро здравствуйте дорогие участн...
1,aJFxUKudzwU,"[Рашид Исмаилов, Мэнди Карвер, Вольфганг Клейн...",Секция 1. Управление интернетом,В июле 2018 года Генеральный секретарь ООН объ...,2021,российский форум по управлению интернетом и се...
2,pdTQahZV5TQ,"[Эльза Ганеева, Андрей Игнатьев, Презентация, ...",Секция 2. Новые технологии. Искусственный инте...,В соответствии с «Концепцией развития регулиро...,2021,секция называется новые технологии искусственн...
3,q4R98A4Eh84,"[Милош Вагнер, Наталья Великородняя, Николай Д...",Секция 3. Суверенитет данных,"Суверенитет данных – это идея о том, что данны...",2021,доброе утро уважаемые участники 11 российского...
4,XQZTCXwWIvE,[],Вручение награды Virtuti Interneti и традицион...,В 2010 г. Координационный центр доменов .RU/.Р...,2021,дорогие друзья мы переходим следующему этапу 1...
...,...,...,...,...,...,...
69,None,[],Круглый стол 1. Российская и европейская поз...,Европа - один признанных лидеров в области ра...,2011,None
70,None,[],Круглый стол 2. Управление интернетом в Росси...,"Модераторы: Маркус Куммер, Вице-президент ISO...",2011,None
71,None,[],Перерыв на кофе,None,2011,None
72,None,[],Сценарии будущего развития интернета: взгляд и...,"Большинство экспертов предполагают, что в буд...",2011,None


In [16]:
df.to_json('all_data.json')

# Статистика

**Всего сессий**

In [17]:
df.size

444

**Всего персон**

In [18]:
import itertools
all_persons = [r['persons'] for r in reports]
all_persons = set(itertools.chain.from_iterable(all_persons))

len(all_persons)


131

**Докладов с транскрипцией**

In [19]:
len(list(filter(lambda r: r['video_transcript'] is not None, reports)))

12

**Граф связей**

In [20]:
persons_to_reports = {}

for report in reports:
    for person in report['persons']:
        if person not in persons_to_reports:
            persons_to_reports[person] = []
        persons_to_reports[person].append((report['year'], report['title']))

**Персоны с наиболее длительным присутcтвием на IGF**

In [21]:
persons_to_year_count = {}
for p, rs in persons_to_reports.items():
    persons_to_year_count[p] = list(set([r[0] for r in rs]))

top_20_long_presence = list(sorted(persons_to_year_count.items(), key=lambda kv: len(kv[1]), reverse=True))[:20]
top_20_long_presence

[('Андрей Ярных', [2016, 2017, 2021, 2013, 2014]),
 ('Олег Демидов', [2016, 2017, 2013]),
 ('Георгий Грицай', [2016, 2013, 2014]),
 ('Руслан Гаттаров', [2012, 2013, 2014]),
 ('Андрей Воробьев', [2021, 2014]),
 ('Вольфганг Клейнвахтер', [2013, 2021]),
 ('Презентация', [2017, 2021]),
 ('Сергей Плуготаренко', [2017, 2021]),
 ('Вольфганг Кляйнвахтер', [2016, 2018]),
 ('Маарит Паловирта', [2016, 2017]),
 ('Роберт Шлегель', [2016, 2012]),
 ('Андрей Колесников', [2012, 2014]),
 ('Марина Никерова', [2013, 2014]),
 ('Ульяна Зинина', [2013, 2014]),
 ('Вени Марковский', [2013, 2014]),
 ('Павел Храмцов', [2013, 2014]),
 ('Мадина Касенова', [2013, 2014]),
 ('Джефф Мосс', [2012, 2013]),
 ('Максут Шадаев', [2021]),
 ('Александр Хинштейн', [2021])]

**Персоны с присутсвием в большом числе докладов**

In [22]:
persons_to_report_count = {}
for p, rs in persons_to_reports.items():
    persons_to_report_count[p] = len(rs)

top_20_report_count = list(sorted(persons_to_report_count.items(), key=lambda kv: kv[1], reverse=True))[:20]
top_20_report_count

[('Презентация', 12),
 ('Андрей Ярных', 7),
 ('Олег Демидов', 6),
 ('Георгий Грицай', 5),
 ('Руслан Гаттаров', 4),
 ('Ульяна Зинина', 4),
 ('Вени Марковский', 4),
 ('Павел Храмцов', 4),
 ('Мадина Касенова', 4),
 ('Бертран де ля Шапель', 4),
 ('Андрей Воробьев', 3),
 ('Вольфганг Клейнвахтер', 3),
 ('Вольфганг Кляйнвахтер', 3),
 ('Аксель Павлик', 3),
 ('Марина Никерова', 3),
 ('Джефф Мосс', 3),
 ('Татьяна Матвеева', 2),
 ('Рашид Исмаилов', 2),
 ('Сергей Плуготаренко', 2),
 ('Маарит Паловирта', 2)]

**Статистика слов из агрегированных докладов по годам**

In [23]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language="russian")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sergei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sergei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [25]:
words_spoken = {}
russian_stopwords = stopwords.words("russian")

for report in reports:
    if report['video_transcript'] is None: continue
    year = report['year']
    if year not in words_spoken: words_spoken[year] = {}
    r_words_spoken = word_tokenize(report['video_transcript'], language="russian")
    
    for w in r_words_spoken:
        w = morph.parse(w)[0].normal_form
        if w in russian_stopwords: continue
        if w not in words_spoken[year]: words_spoken[year][w] = 0
        words_spoken[year][w] += 1

words_spoken.keys()

dict_keys([2021, 2019, 2014])

In [26]:
def top_100_words(items):
    return list(sorted(items, key=lambda wc: wc[1], reverse=True))[:100]

In [33]:
top_2021_words = top_100_words(words_spoken[2021].items())
top_2019_words = top_100_words(words_spoken[2019].items())
top_2014_words = top_100_words(words_spoken[2014].items())

In [28]:
' '.join(w[0] for w in top_2021_words)

'это который всё очень вопрос интернет мочь наш весь говорить свой большой какой-то человек цифровой государство год спасибо платформа должный ещё хотеть компания данные страна право сегодня поэтому сказать управление дело работа первый форум российский собственно сторона дать решение принцип международный просто знать мир правило слово нужно действительно проблема работать наверное россия общий также регулирование интеллект пользователь бизнес развитие возможность уровень система например разный являться технология думать самый именно время понимать участник новый иметь доверие организация точка важно информация сделать день глобальный число достаточно видеть искусственный сеть делать коллега важный тема больший безопасность заниматься зрение процесс соответственно общество вообще хороший'

In [29]:
' '.join(w[0] for w in top_2019_words)

'это который интернет данные очень дать вопрос всё наш говорить человек мочь должный весь сегодня знать управление год свой ещё пользователь спасибо право защита большой какой-то являться хотеть дело самый персональный форум думать ваш сказать разный сторона давать государство процесс первый информация время международный глобальный регулирование закон действительно дискуссия например нужно страна жизнь решение коллега общество понимать развитие поэтому контент россия компания просто pr будущее конвенция робот европа цифровой именно хороший принцип любой новый уровень мир использовать европейский технический китай интересно германия также российский важно необходимо точка подход часть среда фейсбук организация господин китайский использование важный искусственный интеллект принять наверное'

In [34]:
' '.join(w[0] for w in top_2014_words)

'это интернет который очень наш европа совет всё год российский весь хотеть человек сказать говорить мочь домен сегодня спасибо федерация большой важный праздник форум работать слово россия страна управление центр свой конвенция время пожалуйста видеть принять работа первый друг господин коллега директор самый ещё сообщество 20 ваш свобода безопасность отношение 20-летие вещь собственно поздравлять ребёнок международный представитель представить член закон данные хотя заниматься связь регион парламентский ассамблея назад орден присоединиться ru часть поэтому открытие знать достаточно колесников юбилей точка значит . продолжать являться информационный поздравить должный компания персональный мир какой-то отключить вместе технический успешный добрый утро уважаемый действительно особый поскольку'

## Новые слова и тренды

In [30]:
common = set(w[0] for w in top_2021_words) & set(w[0] for w in top_2019_words) 

**2021**

In [31]:
' '.join(w[0] for w in top_2021_words if w[0] not in common)

'платформа работа собственно правило слово проблема работать общий бизнес возможность система технология участник иметь доверие сделать день число достаточно видеть сеть делать тема больший безопасность заниматься зрение соответственно вообще'

**2019**

In [32]:
' '.join(w[0] for w in top_2019_words if w[0] not in common)

'защита персональный ваш давать закон дискуссия жизнь контент pr будущее конвенция робот европа любой использовать европейский технический китай интересно германия необходимо подход часть среда фейсбук господин китайский использование принять'

**2014**

In [35]:
' '.join(w[0] for w in top_2014_words if w[0] not in common)

'европа совет домен федерация праздник работать слово центр конвенция пожалуйста видеть принять работа друг господин директор сообщество 20 ваш свобода безопасность отношение 20-летие вещь собственно поздравлять ребёнок представитель представить член закон хотя заниматься связь регион парламентский ассамблея назад орден присоединиться ru часть открытие достаточно колесников юбилей значит . продолжать информационный поздравить персональный отключить вместе технический успешный добрый утро уважаемый особый поскольку'